# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [1]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [2]:
import random


In [3]:
def count_edges(edges, cut):
    ans = 0
    for edge in edges:
        if not(set(edge) <= cut[0] or set(edge) <= cut[1]):
            ans += 1
    return ans

In [4]:
def find_nearest(edges, cut, k):
    ans = (0, 0)
    best_dist = count_edges(edges, cut)
    tmp = np.zeros(len(cut[0]) + len(cut[1]) + 1)
    tmp[list(cut[1])] = 1
    for first in cut[0]:
        for second in cut[1]:
            sum_first = tmp[list(edges[first])].sum()
            sum_second = tmp[list(edges[second])].sum()
            dist = best_dist + (len(edges[first]) - sum_first) - sum_first + sum_second - (len(edges[second]) - sum_second)
            if dist < best_dist:
                best_dist, ans = dist, (first, second)
    cut[0].add(ans[1])
    cut[0].remove(ans[0])
    cut[1].add(ans[0])
    cut[1].remove(ans[1])
    return [cut[0], cut[1]]

In [5]:
import numpy as np

In [20]:
def variable_depth_local_search(graph):
    vertices = graph[0]
    edges = graph[1]
    new_edges = [[] for i in range(len(graph[0]) + 1)]
    for edge in edges:
        new_edges[edge[0]].append(edge[1])
        new_edges[edge[1]].append(edge[0])
    cut = [set(range(1, len(graph[0]) // 2 + 1)), vertices.difference(set(range(1, len(graph[0]) // 2 + 1)))]
    
    np_cut = np.zeros(len(vertices) + 1)
    np_cut[list(cut[1])] = 1
    
    sum_vert = np.zeros(len(vertices) + 1)
    for v in range(1, len(vertices) + 1):
        sum_vert[v] = np_cut[new_edges[v]].sum()
    cnt_edge = np.zeros(len(vertices) + 1)
    for v in vertices:
        cnt_edge[v] = len(new_edges[v])
    current = count_edges(edges, cut)
    MAX_DEPTH = 1
    while True:
        res = False
        cur_dist = current
        exception = [-1, -2]
        for i in range(MAX_DEPTH):
            ans = [0, 0]
            best_dist = 99999999999999
            for first in cut[0]:
                for second in cut[1]:
                    if({first, second} <= set(exception)):
                        continue
                    sum_first = sum_vert[first]
                    sum_second = sum_vert[second]
                    dist = cur_dist + (cnt_edge[first] - sum_first) - sum_first + sum_second - (cnt_edge[second] - sum_second)
                    if dist < best_dist:
                        best_dist, ans = dist, [first, second]
            cut[0].add(ans[1])
            cut[0].remove(ans[0])
            cut[1].add(ans[0])
            cut[1].remove(ans[1])
            np_cut[ans[1]] = 0
            np_cut[ans[0]] = 1
            sum_vert[new_edges[ans[1]]] -= 1
            sum_vert[new_edges[ans[0]]] += 1
            exception = [ans[0], ans[1]]
            if best_dist < current:
                current = best_dist
                res = True
                break
            else:
                cur_dist = best_dist
        if res == False:
            break
    return cut[0]

In [7]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    for filename in filenames:
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [ ]:
run_all()

Solving instance add20.graph… done in 1.5e+02 seconds with quality 11876
Solving instance cti.graph… done in 3e+02 seconds with quality 94072
Solving instance t60k.graph… done in 3.2e+03 seconds with quality 178248
Solving instance m14b.graph…

## Выводы

Как видим даже для параметра 1 (просто локальный поиск) алгоритм работает очень долго. Как еще ускорить я не знаю, на мой взгляд я предподсчитал все что было возможно, а избавиться от 3 вложенных циклов невозможно.